# Multi-GPU Mean Calculation

In [203]:
from dask.distributed import Client

import cudf, dask_cudf
from dask_cuml import mean

There are a couple ways to get data into cuml, which will need to be tested:
1. A large cudf object could be created and then passed to dask_cudf
2. The workers are asked to fetch the data directly

Since this will likely be running in a single worker per GPU mode, it will be important that the cuDF's are able to work across the GPUs (e.g. When a very large cuDF is partitioned across the workers- it will be important that the GPU memory is re-allocated to the new worker's local device and de-allocated on the cuDF's old device.)

__Example workflow__:
- User allocates a dask_cudf (or, eventually, a dask_cuml_array) and distributes it across the cluster
- User calls MGMean().calculate(dask_cudf) after the dask_cudf
- MGMean performs redistribution / preprocessing
- MGMean gathers allocations (hostname/device/key triplets) from Dask workers
- MGMean c++ code is executed with the allocation information as its argument


In [204]:
client = Client("10.31.241.47:8786")

In [205]:
client

Client Scheduler: tcp://10.31.241.47:8786,Cluster Workers: 5 Cores: 5 Memory: 33.81 GB


In [206]:
client.who_has()

{}

In [207]:
def create_cudf(dev):
    import numba.cuda
    import numpy as np
    numba.cuda.select_device(dev)
    print("Creating dataframe on device " + str(dev))
    return (dev, cudf.DataFrame(
        [('a', np.array([1.0, 2.0, 3.0, 4.0, 5.0, 6.0]).astype(np.float32)), 
         ('b', np.array([2.0, 3.0, 4.0, 5.0, 6.0, 7.0]).astype(np.float32))]
    ))

In [208]:
workers = list(client.has_what().keys())
workers

['tcp://10.31.241.47:33516',
 'tcp://10.31.241.47:41165',
 'tcp://10.31.241.47:35201',
 'tcp://10.31.241.47:46597',
 'tcp://10.31.241.47:38489']

In [332]:
# Copyright (c) 2018, NVIDIA CORPORATION.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#
from cuml import MGMean as cumlMGMean

from tornado import gen
import dask_cudf, cudf

import time

from dask.distributed import get_worker, get_client

from dask import delayed
from collections import defaultdict
from dask.distributed import wait, default_client
import dask.dataframe as dd
import dask.array as da

from toolz import first, assoc
from distributed import Client


def parse_host_port(address):
    if '://' in address:
        address = address.rsplit('://', 1)[1]
    host, port = address.split(':')
    port = int(port)
    return host, port

def to_gpu_matrix(inp):
    dev, df = inp
    import numba.cuda
    numba.cuda.select_device(dev)
    print(str(numba.cuda.get_current_device()) + " - " + str(dev))
    rm = df.as_gpu_matrix(order='F')
    print("GPU: " + str(rm))
    print("CTYPES: "+ str(rm.device_ctypes_pointer))
    series = build_output_series(rm, dev)
    return (dev, rm, series, to_gpu_array(series))

def build_output_series(gpu_matrix, dev):
    import numpy as np
    import numba.cuda
    numba.cuda.select_device(dev)
    print(str(numba.cuda.get_current_device()) + " - " + str(dev))
    return cudf.Series(np.zeros(gpu_matrix.shape[1], dtype=gpu_matrix.dtype))

def to_gpu_array(mean_):
    return mean_._column._data.to_gpu_array()

from threading import Thread
import numba.cuda
import time

from threading import Lock

class IPCThread(Thread):
    
    def __init__(self, in_ipc, out_ipc, dev):
        Thread.__init__(self)

        self.lock = Lock()
        self.in_ipc = in_ipc
        self.out_ipc = out_ipc
        self.dev = dev
        self.running = False
        
        
    def run(self):
        
        numba.cuda.select_device(self.dev)
        print("Opening: " + str(self.dev) + str(numba.cuda.get_current_device()))
        
        self.lock.acquire()
        try:
            self.in_arr = self.in_ipc.open()
            self.out_arr = self.out_ipc.open()
        except Exception as e:
            print("Error opening ipc_handle on device " + str(self.dev) + ": " + str(e))
        
        cai = self.in_arr.__cuda_array_interface__
        self.ptr_info = {"ptr": self.in_arr.device_ctypes_pointer.value,
                "out_ptr": self.out_arr.device_ctypes_pointer.value,
                "dtype": cai["typestr"],
                "shape": cai["shape"]
        }
        
        self.running = True
        self.lock.release()

        while(self.running):
            time.sleep(0.0001)
        try:
            print("Closing: " + str(self.dev) + str(numba.cuda.get_current_device()))

            self.lock.acquire()
            self.in_ipc.close()
            self.out_ipc.close()
            self.lock.release()
        except Exception as e:
            print("Error closing ipc_handle on device " + str(self.dev) + ": " + str(e))


    def close(self):
        
        self.lock.acquire()
        self.running = False
        self.lock.release()
        
    def info(self):
        
        while(not self.running):
            time.sleep(0.0001)
            
        return self.ptr_info

    
def build_alloc_info(data):
    dev, gpu_matrix, series, gpu_array = data
    
    cai = gpu_matrix.__cuda_array_interface__
    return {"ptr": gpu_matrix.device_ctypes_pointer.value,
        "out_ptr": gpu_array.device_ctypes_pointer.value,
        "dtype": cai["typestr"],
        "shape": cai["shape"]
    }
    

def get_ipc_handles(data):
    
    import numba.cuda
    print("CURRENT DEVICE: " + str(numba.cuda.get_current_device()))
    dev, gpu_matrix, series, gpu_array = data
    
    try:
        in_handle = gpu_matrix.get_ipc_handle()
        out_handle = gpu_matrix.get_ipc_handle()
        
        return (dev, gpu_matrix.get_ipc_handle(), gpu_array.get_ipc_handle())
    except Exception as e:
        print("Error: " + str(e))
        return (dev, None, None)


# Run on a single worker on each unique host
def calc_mean(data):

    import numba.cuda
    print("calc_mean_device: " + str(numba.cuda.get_current_device()))
    
    ipcs, raw_arrs = data

    def get_ipc_thread(dev, inp, outp):
        t = IPCThread(inp, outp, dev)
        t.start()
        return t

    open_ipcs = [get_ipc_thread(dev, inp, outp) for dev, inp, outp in ipcs]
    print("calc_mean_device: " + str(numba.cuda.get_current_device()))
    m = cumlMGMean()
    
    alloc_info = [t.info() for t in open_ipcs]
    alloc_info.extend([build_alloc_info(t) for t in raw_arrs])
    
    print("calc_mean_device: " + str(numba.cuda.get_current_device()))
    m.calculate(alloc_info)

    print("calc_mean_device: " + str(numba.cuda.get_current_device()))
    return open_ipcs, raw_arrs
    
class MGMean(object):

    def calculate(self, futures):
        client = default_client()

        # Keep the futures around so the GPU memory doesn't get
        # deallocated on the workers.
        gpu_futures = client.sync(self._get_mg_info, futures)

        who_has = client.who_has(gpu_futures)

        key_to_host_dict = {}
        for key in who_has:
            key_to_host_dict[key] = parse_host_port(who_has[key][0])
            
        hosts_to_key_dict = {}
        for key, host in key_to_host_dict.items():
            if host not in hosts_to_key_dict:
                hosts_to_key_dict[host] = set([key])
            else:
                hosts_to_key_dict[host].add(key)

        workers = [key[0] for key in list(who_has.values())]
        hosts_dict = build_host_dict(workers)
        f = []
        for host, ports in hosts_dict.items():
            exec_node = (host, random.sample(ports, 1)[0])
            
            print("Chosen exec node is "  + str(exec_node))
            
            # Don't build an ipc_handle for exec nodes (we can just grab the local data)
            keys = set(hosts_to_key_dict[exec_node])
            
            # build ipc handles
            gpu_data_excl_worker = filter(lambda d: d[0] != exec_node, gpu_futures)
            gpu_data_incl_worker = filter(lambda d: d[0] == exec_node, gpu_futures)
            
            ipc_handles = [client.submit(get_ipc_handles, future, workers=[worker])
                           for worker, future in gpu_data_excl_worker]
            raw_arrays = [future for worker, future in gpu_data_incl_worker]
            
            print(str(ipc_handles))
            print(str(raw_arrays))
            
            f.append(client.submit(calc_mean, (ipc_handles, raw_arrays), workers = [exec_node]))

        wait(f)
        
        def close_threads(d):
            print(str("Closing threads!"))
            ipc_threads, rawarrays = d
            [t.close() for t in ipc_threads]
            
        d = [client.submit(close_threads, future) for future in f]
        wait(d)
        
        def join_threads(d):
            print(str("Joining threads!"))
            ipc_threads, rawarrays = d
            [t.join() for t in ipc_threads]
            
        d = [client.submit(join_threads, future) for future in f]
        
        def print_it(data):
            dev, gpu_mat, series, gpu_arr = data
            return str(series)

        return client.gather([client.submit(print_it, future) for worker, future in gpu_futures])
    

    @gen.coroutine
    def _get_mg_info(self, futures):

        client = default_client()

        if isinstance(futures, dd.DataFrame):
            data_parts = futures.to_delayed()
            parts = list(map(delayed, data_parts))
            parts = client.compute(parts)  # Start computation in the background
            yield wait(parts)
            for part in parts:
                if part.status == 'error':
                    yield part  # trigger error locally
        else:
            data_parts = futures

        key_to_part_dict = dict([(str(part.key), part) for part in data_parts])

        who_has = yield client.who_has(data_parts)
        worker_map = []
        for key, workers in who_has.items():
            
            worker = parse_host_port(first(workers))
            worker_map.append((worker, key_to_part_dict[key]))

        gpu_data = [(worker, client.submit(to_gpu_matrix, part, workers=[worker]))
                    for worker, part in worker_map]
        
        yield wait(gpu_data)

        raise gen.Return(gpu_data)

In [333]:
from dask.distributed import wait
import random

def parse_host_port(address):
    if '://' in address:
        address = address.rsplit('://', 1)[1]
    host, port = address.split(':')
    port = int(port)
    return host, port

def build_host_dict(workers):
    hosts = set(map(lambda x: parse_host_port(x), workers))
    hosts_dict = {}
    for host, port in hosts:
        if host not in hosts_dict:
            hosts_dict[host] = set([port])
        else:
            hosts_dict[host].add(port)
            
    return hosts_dict
    

def assign_gpus(client):
    
    """
    Supports a multi-GPU & multi-Node environment by assigning a single local GPU
    to each worker in the cluster. This is necessary due to Numba's restriction
    that only a single CUDA context (and thus a single device) can be active on a 
    thread at a time. 
    
    The GPU assignments are valid as long as the future returned from this function
    is held in scope. This allows any functions that need to allocate GPU data to
    utilize the CUDA context on the same device, otherwise data could be lost.
    """

    workers = list(client.has_what().keys())
    hosts_dict = build_host_dict(workers)
    
    def get_gpu_info():
        import numba.cuda
        return [x.id for x in numba.cuda.gpus]
    
    gpu_info = dict([(host, 
                      client.submit(get_gpu_info, 
                                    workers = [(host, random.sample(hosts_dict[host], 1)[0])])) 
                     for host in hosts_dict])
    wait(list(gpu_info.values()))
    
    # Scatter out a GPU device ID to workers
    f = []
    for host, future in gpu_info.items():
        gpu_ids = future.result()
        ports = random.sample(hosts_dict[host], min(len(gpu_ids), len(hosts_dict[host])))
        
        f.extend([client.scatter(device_id, workers = [(host,port)]) for device_id, port in zip(gpu_ids, ports)])
    wait(f)
        
    return f

In [334]:
assignments = assign_gpus(client)

In [335]:
client.who_has()

{'calc_mean-d14aa6846e36c0b876d1e67ce6c69330': ('tcp://10.31.241.47:41165',),
 'create_cudf-40ef018e7d18ab18a969bc2f9f3e32cd': ('tcp://10.31.241.47:41165',),
 'create_cudf-5917e56cda471a44f4834546bfcac9ba': ('tcp://10.31.241.47:38489',),
 'create_cudf-8d78d86a7729d3150eb1b8f7b0cc4c4d': ('tcp://10.31.241.47:46597',),
 'create_cudf-96c79b3959983f80afb0e61ab1df27a1': ('tcp://10.31.241.47:35201',),
 'create_cudf-e2fc6683845ae641dac2c0a379b81ca2': ('tcp://10.31.241.47:33516',),
 'get_ipc_handles-28ff925961d748ead9ffcdda6781f9d9': ('tcp://10.31.241.47:35201',
  'tcp://10.31.241.47:41165'),
 'get_ipc_handles-3e3a0679c38607409c418eb75840639a': ('tcp://10.31.241.47:33516',
  'tcp://10.31.241.47:41165'),
 'get_ipc_handles-47071f2913142f39eb4c2d6526eb10a5': ('tcp://10.31.241.47:38489',
  'tcp://10.31.241.47:41165'),
 'get_ipc_handles-508189c060da94aebb9bb373d9fc86f7': ('tcp://10.31.241.47:46597',
  'tcp://10.31.241.47:41165'),
 'int-58e78e1b34eb49a68c65b54815d1b158': ('tcp://10.31.241.47:35201',


In [336]:
res = [client.submit(create_cudf, future, workers = [worker]) for future, worker in zip(assignments, workers)]
wait(res)

DoneAndNotDoneFutures(done={<Future: status: finished, type: tuple, key: create_cudf-40ef018e7d18ab18a969bc2f9f3e32cd>, <Future: status: finished, type: tuple, key: create_cudf-e2fc6683845ae641dac2c0a379b81ca2>, <Future: status: finished, type: tuple, key: create_cudf-5917e56cda471a44f4834546bfcac9ba>, <Future: status: finished, type: tuple, key: create_cudf-8d78d86a7729d3150eb1b8f7b0cc4c4d>, <Future: status: finished, type: tuple, key: create_cudf-96c79b3959983f80afb0e61ab1df27a1>}, not_done=set())

In [337]:
client.who_has()

{'calc_mean-d14aa6846e36c0b876d1e67ce6c69330': ('tcp://10.31.241.47:41165',),
 'create_cudf-40ef018e7d18ab18a969bc2f9f3e32cd': ('tcp://10.31.241.47:41165',),
 'create_cudf-5917e56cda471a44f4834546bfcac9ba': ('tcp://10.31.241.47:38489',),
 'create_cudf-8d78d86a7729d3150eb1b8f7b0cc4c4d': ('tcp://10.31.241.47:46597',),
 'create_cudf-96c79b3959983f80afb0e61ab1df27a1': ('tcp://10.31.241.47:35201',),
 'create_cudf-e2fc6683845ae641dac2c0a379b81ca2': ('tcp://10.31.241.47:33516',),
 'get_ipc_handles-28ff925961d748ead9ffcdda6781f9d9': ('tcp://10.31.241.47:35201',
  'tcp://10.31.241.47:41165'),
 'get_ipc_handles-3e3a0679c38607409c418eb75840639a': ('tcp://10.31.241.47:33516',
  'tcp://10.31.241.47:41165'),
 'get_ipc_handles-47071f2913142f39eb4c2d6526eb10a5': ('tcp://10.31.241.47:38489',
  'tcp://10.31.241.47:41165'),
 'get_ipc_handles-508189c060da94aebb9bb373d9fc86f7': ('tcp://10.31.241.47:46597',
  'tcp://10.31.241.47:41165'),
 'int-58e78e1b34eb49a68c65b54815d1b158': ('tcp://10.31.241.47:35201',


In [338]:
m = MGMean()

In [341]:
m.calculate(res)

Chosen exec node is ('10.31.241.47', 38489)
[<Future: status: pending, key: get_ipc_handles-f2934afc1fb9dc757d6ddf4b7d153c87>, <Future: status: pending, key: get_ipc_handles-6cc70fec43102542ddf7b7cb1555c422>, <Future: status: pending, key: get_ipc_handles-46c1e557a8bed5898d4803721d0e8e1d>, <Future: status: pending, key: get_ipc_handles-2597a091f21738f85c10689df966fccb>]
[<Future: status: finished, type: tuple, key: to_gpu_matrix-8965f172b71e33b21bbaa01a9b8ce0d1>]


['      \n0  3.5\n1  4.5',
 '      \n0  3.5\n1  4.5',
 '      \n0  3.5\n1  4.5',
 '      \n0  3.5\n1  4.5',
 '      \n0  3.5\n1  4.5']

In [342]:
client.who_has()

{'calc_mean-d14aa6846e36c0b876d1e67ce6c69330': ('tcp://10.31.241.47:41165',),
 'create_cudf-40ef018e7d18ab18a969bc2f9f3e32cd': ('tcp://10.31.241.47:41165',),
 'create_cudf-5917e56cda471a44f4834546bfcac9ba': ('tcp://10.31.241.47:38489',),
 'create_cudf-8d78d86a7729d3150eb1b8f7b0cc4c4d': ('tcp://10.31.241.47:46597',),
 'create_cudf-96c79b3959983f80afb0e61ab1df27a1': ('tcp://10.31.241.47:35201',),
 'create_cudf-e2fc6683845ae641dac2c0a379b81ca2': ('tcp://10.31.241.47:33516',),
 'get_ipc_handles-28ff925961d748ead9ffcdda6781f9d9': ('tcp://10.31.241.47:35201',
  'tcp://10.31.241.47:41165'),
 'get_ipc_handles-3e3a0679c38607409c418eb75840639a': ('tcp://10.31.241.47:33516',
  'tcp://10.31.241.47:41165'),
 'get_ipc_handles-47071f2913142f39eb4c2d6526eb10a5': ('tcp://10.31.241.47:38489',
  'tcp://10.31.241.47:41165'),
 'get_ipc_handles-508189c060da94aebb9bb373d9fc86f7': ('tcp://10.31.241.47:46597',
  'tcp://10.31.241.47:41165'),
 'int-58e78e1b34eb49a68c65b54815d1b158': ('tcp://10.31.241.47:35201',


In [323]:
print(str(result))

NameError: name 'result' is not defined

In [ ]:
client.who_has()

